# Import Required Libraries
Import the necessary libraries, including pandas, NumPy, TensorFlow, and Keras.

In [2]:
# Importing the required libraries

# Pandas is used for data manipulation and analysis

# NumPy is used for numerical operations
import numpy as np

# Keras is a user-friendly neural network library written in Python
from tensorflow import keras

# Load and Inspect the Data
Load the training.csv file into a DataFrame and inspect its structure and contents.

In [ ]:
import tensorflow as tf

# Define a function to parse the tfrecords file
def _parse_function(proto):
    # Define your tfrecord columns
    columns = {
        'detectorPositionX': tf.io.VarLenFeature(tf.float32),
        'detectorPositionY': tf.io.VarLenFeature(tf.float32),
        'duplicates_count': tf.io.VarLenFeature(tf.float32),
        'label': tf.io.FixedLenFeature([], tf.float32)
    }
    
    # Parse the input tf.Example proto using the dictionary above.
    parsed_features = tf.io.parse_single_example(proto, columns)

    # Convert SparseTensors to dense
    x = tf.sparse.to_dense(parsed_features['detectorPositionX'])
    y = tf.sparse.to_dense(parsed_features['detectorPositionY'])
    duplicates_count = tf.sparse.to_dense(parsed_features['duplicates_count'])

    # Flatten x, y and duplicates_count
    x = tf.reshape(x, [-1])
    y = tf.reshape(y, [-1])
    duplicates_count = tf.reshape(duplicates_count, [-1])

    # Now you can perform operations on x, y and duplicates_count
    indices = tf.stack([tf.cast((x+680)/8.5, tf.int32) - 1, tf.cast((y+680)/8.5, tf.int32) - 1], axis=1)

    # Create an empty image
    image = tf.zeros((156, 156), dtype=tf.float32)
    updates = duplicates_count
    image = tf.tensor_scatter_nd_update(image, indices, updates)
    image = tf.reshape(image, [156, 156, 1])

    # Return the image and the label
    return image, parsed_features['label']

# Read the tfrecords file
dataset = tf.data.TFRecordDataset( ["groups{}.tfrecords".format(i) for i in range(0, 50)])

# Map the parse function to each record
dataset = dataset.map(_parse_function)

# Now, dataset is a tf.data.Dataset of (image, label) pairs ready for training.

# Prepare the Data
Separate the original DataFrame into training, validation, and test sets. Combine the detectorPositionX and detectorPositionY columns into a single array of dimension 39*39. Separate the images taking into account the eventNumber column.

In [4]:
# Count the total number of examples in the dataset
total_size = sum(1 for _ in dataset)

# Calculate the size of each subset
train_size = int(0.98 * total_size)
val_size = int(0.01 * total_size)  # validation size
test_size = total_size - train_size - val_size  # test size

dataset = dataset.shuffle(32*10)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

# Split the dataset
train_dataset = dataset.take(train_size).batch(32)
val_dataset = dataset.skip(val_size).batch(32)
"""
val_dataset = test_dataset.skip(val_size).batch(64)
test_dataset = test_dataset.take(test_size).batch(64)"""

'\nval_dataset = test_dataset.skip(val_size).batch(64)\ntest_dataset = test_dataset.take(test_size).batch(64)'

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


# Create the Convolutional Neural Network
Create a CNN model with 3 convolutional layers, two dense layers, and a final layer of one neuron to predict the value of the energyPrimary.

In [8]:
# Importing the required layers for the Convolutional Neural Network
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, add, Activation, Input, concatenate
from tensorflow.keras import mixed_precision, Model, regularizers
from tensorflow.keras.initializers import HeNormal

# Define the L2 regularizer
l2_reg = regularizers.l2(0.001)

# Set the policy to 'mixed_float16'
mixed_precision.set_global_policy('mixed_float16')

# Define the He Normal initializer
he_normal_initializer = HeNormal()

def inception_module(x, filters):
    # 1x1 conv
    conv1 = Conv2D(filters*.25, (1, 1), padding='same', activation='relu', kernel_initializer=he_normal_initializer)(x)

    # 1x1 conv -> 3x3 conv
    conv2 = Conv2D(filters, (1, 1), padding='same', activation='relu', kernel_initializer=he_normal_initializer)(x)
    conv2 = Conv2D(filters*.5, (3, 3), padding='same', activation='relu', kernel_initializer=he_normal_initializer)(conv2)

    # 1x1 conv -> 5x5 conv
    conv3 = Conv2D(filters, (1, 1), padding='same', activation='relu', kernel_initializer=he_normal_initializer)(x)
    conv3 = Conv2D(filters*.25, (5, 5), padding='same', activation='relu', kernel_initializer=he_normal_initializer)(conv3)

    return concatenate([conv1, conv2, conv3], axis=-1)

# Define the input shapfilters
input_layer = Input(shape=(156, 156, 1))

# Add the new Conv2D layers
x = Conv2D(16, (3, 3), strides=(2, 2), activation='relu', kernel_initializer=he_normal_initializer)(input_layer)
x = Conv2D(16, (3, 3), strides=(2, 2), activation='relu', kernel_initializer=he_normal_initializer)(x)

x = inception_module(x, 16)
x = inception_module(x, 32)

x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', kernel_initializer=he_normal_initializer)(x)

x = inception_module(x, 64)

x = Conv2D(64, (3, 3), strides=(2, 2), activation='relu', kernel_initializer=he_normal_initializer)(x)

x = Dropout(0.1)(x)

x = Flatten()(x)

# Add your dense layers here
# For example:
x = Dense(128, activation='relu', kernel_initializer=he_normal_initializer)(x)
x = Dense(1, dtype=tf.float32, kernel_initializer=he_normal_initializer)(x)

model = Model(inputs=input_layer, outputs=x)

# Print the summary of the model to check the number of parameters and the structure
model.summary()
# Print the summary of the model to check the number of parameters and the structure

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 156, 156, 1)]        0         []                            
                                                                                                  
 conv2d_25 (Conv2D)          (None, 156, 156, 8)          80        ['input_2[0][0]']             
                                                                                                  
 conv2d_26 (Conv2D)          (None, 156, 156, 8)          584       ['conv2d_25[0][0]']           
                                                                                                  
 conv2d_27 (Conv2D)          (None, 156, 156, 8)          584       ['conv2d_26[0][0]']           
                                                                                            

# Train the Convolutional Neural Network
Train the CNN model using the prepared training data.

In [9]:
# Importing the required library for the model compilation and training
from tensorflow.keras.optimizers import Adam

def mean_absolute_log_error(y_true, y_pred):
    return tf.reduce_mean(tf.math.abs(tf.math.log1p(y_true) - tf.math.log1p(y_pred)))

# Compile the model
# We use the Adam optimizer, Mean Squared Logarithmic Error (MSLE) as loss function, and Mean Absolute Percentage Error (MAPE) as metric

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_percentage_error'])

# Train the model for 10 epochs using the training and validation data
# We use a batch size of 32
history = model.fit(train_dataset, epochs=5, validation_data=val_dataset, verbose=1)

# Save the trained model for future use
model.save('cnn_model.h5')

Epoch 1/5
     12/Unknown - 60s 5s/step - loss: 7.5318 - mean_absolute_percentage_error: 426.4277

KeyboardInterrupt: 

# Evaluate the Model
Evaluate the performance of the trained model using the test data.

In [ ]:
# Importing the required library for visualizing the model's performance
import matplotlib.pyplot as plt

# Evaluate the model using the test data
test_loss, test_mae = model.evaluate(test_images, test_labels)

# Print the test loss and test mean absolute error
print('Test loss:', test_loss)
print('Test MAE:', test_mae)

# Plot the training and validation loss over each epoch
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the training and validation mean absolute error over each epoch
plt.figure(figsize=(12, 6))
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Training and Validation MAE Over Time')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()